In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 32.7 MB/s 
     |████████████████████████████████| 596 kB 24.8 MB/s 
     |████████████████████████████████| 86 kB 6.4 MB/s 
     |████████████████████████████████| 6.6 MB 54.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import random
import pickle
import os
import nltk
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import tensorflow as tf 
import string
import re
import pandas as pd 
from collections import Counter
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize, pos_tag
from sklearn.model_selection import train_test_split
from google.colab import drive
import nltk
nltk.download('omw-1.4')
drive.mount('/content/drive')
!ls '/content/drive/Shareddrives/Deep Learning/DeepLearning_2022/Final project/Data/'
myDrive = '/content/drive/Shareddrives/Deep Learning/DeepLearning_2022/Final project/Data/'
results_path = '/content/drive/Shareddrives/Deep Learning/DeepLearning_2022/Final project/Results/'

Mounted at /content/drive
 all_words.txt	 data_clean.csv		 'IMDB Dataset.csv'   sorted_words.txt
 data_1		 data_no_stop_words.csv   inputs_ids.pt


In [ ]:
data = pd.read_csv(f'{myDrive}data_clean.csv')
file1 = open(f'{myDrive}all_words.txt', 'r')
file2 = open(f'{myDrive}sorted_words.txt', 'r')
all_words = file1.read().splitlines()
count_words = Counter(all_words)
sorted_words=count_words.most_common(len(all_words))
vocab_to_int={w:i+1 for i,(w,c) in enumerate(sorted_words)}


In [ ]:
data.sentiment = [0 if each == "negative" else 1 for each in data.sentiment]

In [ ]:
def encode_review(review):
  encoded_review=list()
  words = review.split(' ')
  for word in words:
    encoded_review.append(vocab_to_int[word])

  return encoded_review

In [ ]:
data['review'] = data.apply(lambda row: encode_review(row['review']), axis = 1)
data['review_length'] = data.apply(lambda row: len(row['review']), axis = 1)

The sequence length of our sentences should be the same. Thus, we will calculate the average length of reviews (in words) and truncate or pad each review to have such amount of words. 

In [ ]:
sequence_length = int(sum(list(data['review_length']))/data.shape[0]) # average length
sequence_length_max = max(list(data['review_length']))
print(sequence_length)

def truncate_or_pad(review):
  num_words = len(review)
  if num_words<sequence_length:
    zeros = list(np.zeros(sequence_length-num_words))
    new = zeros+review
  else:
    new = review[:sequence_length]

  return new

def pad(review):
  num_words = len(review)
  new = review
  if num_words<sequence_length_max:
    zeros = list(np.zeros(sequence_length_max-num_words))
    new = zeros+review
  return new

#test = truncate_or_pad(data.review[0])
data_pad = data.copy()
data['review'] = data.apply(lambda row: truncate_or_pad(row['review']), axis = 1)
data_pad['review'] = data_pad.apply(lambda row: pad(row['review']), axis = 1)

118


In [ ]:
'''data_matrix = np.zeros((data.shape[0], sequence_length), dtype=int)
for i in range(data.shape[0]):
  data_matrix[i, :] = np.array(data.review[i])
print(data_matrix[0:15, :])
print(data.review_length[2])'''

[[    3  1020   326 ...    23  1356    12]
 [    0     0     0 ...  1816  1676    19]
 [    0     0     0 ...    13     6   122]
 ...
 [ 3771   105   110 ...   631     2   546]
 [    0     0     0 ...    23    13   341]
 [    0     0     0 ...  1104 11136  6458]]
85


In [ ]:
data_matrix_pad = np.zeros((data_pad.shape[0], sequence_length_max), dtype=int)
for i in range(data_pad.shape[0]):
  data_matrix_pad[i, :] = np.array(data_pad.review[i])
print(data_matrix_pad[0:15, :])
print(data_pad.review_length[2])

[[    0     0     0 ...   469  3431   323]
 [    0     0     0 ...  1816  1676    19]
 [    0     0     0 ...    13     6   122]
 ...
 [    0     0     0 ...   179    90   160]
 [    0     0     0 ...    23    13   341]
 [    0     0     0 ...  1104 11136  6458]]
85


# Bert Word Embedding and model 

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tf_batch = tokenizer(list(data2['review']), max_length=128, padding=True, truncation=True, return_tensors='tf')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def tokenize(review):
  review = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(review))
  return review

In [ ]:
data2['review'] = data2.apply(lambda row: tokenize(row['review']), axis = 1)

In [ ]:
inputs_ids = []
attention_masks = []
for review in data2['review']:
    encoded_dict = tokenizer.encode_plus(
                          review, 
                          add_special_tokens = True, 
                          max_length = sequence_length_max, 
                          pad_to_max_length = True, 
                          return_attention_mask = True,)

    inputs_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

inputs_ids = tf.convert_to_tensor(inputs_ids)
attention_masks = tf.convert_to_tensor(attention_masks)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
def mask_inputs_for_bert(reviews, sequence_length_max, inputs_ids, attention_masks):
  for review in reviews:
    encoded_dict = tokenizer.encode_plus(
                          review, 
                          add_special_tokens = True, 
                          max_length = sequence_length_max, 
                          pad_to_max_length = True, 
                          return_attention_mask = True,)

    inputs_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

  inputs_ids = tf.convert_to_tensor(inputs_ids)
  attention_masks = tf.convert_to_tensor(attention_masks)
  return inputs_ids, attention_masks

In [ ]:
print(inputs_ids)

tf.Tensor(
[[  101  2028 12027 ...     0     0     0]
 [  101  6919  2210 ...     0     0     0]
 [  101  2228  6919 ...     0     0     0]
 ...
 [  101  3234  4036 ...     0     0     0]
 [  101  2175 21090 ...     0     0     0]
 [  101  2028  5987 ...     0     0     0]], shape=(50000, 1418), dtype=int32)


In [ ]:
torch.save(inputs_ids, f'{myDrive}inputs_ids.pt') 

In [ ]:
inputs_ids = torch.load(f'{myDrive}inputs_ids.pt')

# Create and train LSTM model 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_matrix_pad, list(data_pad.sentiment), test_size=0.3)

In [ ]:
train_data=TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
test_data=TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))

In [ ]:
batch_size=50
train_loader=DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader=DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [ ]:
 
class SentimentalLSTM(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):    
        """
        Initialize the model by setting up the layers
        """
        super().__init__()
        self.output_size=output_size
        self.n_layers=n_layers
        self.hidden_dim=hidden_dim
        
        #Embedding and LSTM layers
        self.embedding=nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight = nn.Parameter(torch.tensor(inputs_ids.numpy(),dtype=torch.float32))

        self.lstm=nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        
        #dropout layer
        self.dropout=nn.Dropout(0.5)
        
        #Linear and sigmoid layer
        self.fc1=nn.Linear(hidden_dim, 64)
        self.fc2=nn.Linear(64, 16)
        self.fc3=nn.Linear(16,output_size)
        self.sigmoid=nn.Sigmoid()
        
    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size=x.size()
        
        #Embadding and LSTM output
        embedd=self.embedding(x)
        lstm_out, hidden=self.lstm(embedd, hidden)
        
        #stack up the lstm output
        lstm_out=lstm_out.contiguous().view(-1, self.hidden_dim)
        
        #dropout and fully connected layers
        out=self.dropout(lstm_out)
        out=self.fc1(out)
        out=self.dropout(out)
        out=self.fc2(out)
        out=self.dropout(out)
        out=self.fc3(out)
        sig_out=self.sigmoid(out)
        
        sig_out=sig_out.view(batch_size, -1)
        sig_out=sig_out[:, -1]
        
        return sig_out, hidden
    
    def init_hidden(self, batch_size):
        """Initialize Hidden STATE"""
        # Create two new sensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        return hidden

In [ ]:
def train_loop(model, model_name = 'model.ckpt', device = 'cuda'):
    model.train()
    total_step = len(train_loader)
    losses_list = []
    criterion = nn.BCELoss()

    for epoch in range(num_epochs):
        loss_avg = 0
        h = net.init_hidden(batch_size)
        nBatches = 0
        # TRAINING LOOP
        for i, (review, sentiment) in enumerate(train_loader):
            review = review.type(torch.LongTensor).to(device)
            sentiment = sentiment.to(device)

            h = tuple([each.data for each in h])
            net.zero_grad()

            # Forward pass
            outputs,h = model(review, h)
            # Calculate CE_loss
            loss = criterion(outputs, sentiment)

            # Backpropagate
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_avg += loss.cpu().item()
            nBatches+=1
            if (i+1) % 200 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, i+1, total_step, loss_avg / nBatches))
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, i+1, total_step, loss_avg / nBatches))
        losses_list.append(loss_avg / nBatches)
        torch.save(model.state_dict(), results_path+model_name)


In [ ]:
#Load and test function
def load_and_test(model, device='cuda', model_name = 'model.ckpt'):
  model.load_state_dict(torch.load(results_path+model_name))
  criterion = nn.BCELoss()
  h = net.init_hidden(batch_size)
  # Test the model
  model.eval() # Set the model in evaluation mode

  # Compute testing accuracy
  with torch.no_grad():
      correct = 0
      total = 0
      for review, sentiment in test_loader:
          h = tuple([each.data for each in h])

          review = review.type(torch.LongTensor).to(device)
          sentiment = sentiment.to(device)
          # get network predictions
          outputs, h = model(review, h)

          # get predicted class
          predicted = torch.round(outputs)
          # compare with the ground-truth
          total += sentiment.size(0)
          correct += (predicted == sentiment).sum().item()

      print('Test Accuracy of the model: {} %'.format(100 * correct / total))

In [ ]:
'''# Instantiate the model w/ hyperparams
vocab_size = word_embedding.shape[0]
output_size = 1
embedding_dim = word_embedding.shape[1]
hidden_dim = 128
n_layers = 2

#net = SentimentalLSTM(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
#net = net.cuda()
#print(net)'''

In [ ]:
# Instantiate the model w/ hyperparams
vocab_size = inputs_ids.shape[0]
output_size = 1
embedding_dim = inputs_ids.shape[1]
hidden_dim = 128
n_layers = 2
num_epochs = 6
net = SentimentalLSTM(vocab_size, output_size, embedding_dim, hidden_dim, n_layers).cuda()

optimizer = torch.optim.Adam(net.parameters(),lr = 0.001)

# Device configuration (choose GPU if it is available )
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

train_loop(net, model_name = 'model_4.ckpt')
load_and_test(net,  model_name = 'model_4.ckpt')


In [ ]:
num_epochs = 6
for lr_i in [0.001, 0.0005, 0.002]:
  net = SentimentalLSTM(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
  net = net.cuda()
  optimizer = torch.optim.Adam(net.parameters(),lr = lr_i)

  # Device configuration (choose GPU if it is available )
  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

  train_loop(net, model_name = 'model_4.ckpt')
  load_and_test(net,  model_name = 'model_4.ckpt')

In [ ]:
for lr_i in [0.001, 0.01, 0.05, 0.06]:
  net = SentimentalLSTM(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
  net = net.cuda()
  optimizer = torch.optim.SGD(net.parameters(), lr=lr_i, momentum=0.9)

  # Device configuration (choose GPU if it is available )
  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

  train_loop(net, model_name = 'model_4.ckpt')
  load_and_test(net,  model_name = 'model_4.ckpt')

In [ ]:
for mom in [0.8, 0.85, 0.89, 0.95]:
  net = SentimentalLSTM(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
  net = net.cuda()
  optimizer = torch.optim.SGD(net.parameters(), lr=0.05, momentum=mom)

  # Device configuration (choose GPU if it is available )
  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

  train_loop(net, model_name = 'model_4.ckpt')
  load_and_test(net,  model_name = 'model_4.ckpt')